KPI: Ventas totales y ticket promedio por tienda y mes

In [0]:
CREATE VIEW webinar.gold.ventas_totales
AS
SELECT
  t.id_tienda,
  t.nombre,
  date_format(v.fecha_venta, 'yyyy-MM') AS anio_mes,
  SUM(v.monto)                                    AS ventas_totales,
  COUNT(DISTINCT v.id_venta)                      AS n_transacciones,
  SUM(v.monto) / COUNT(DISTINCT v.id_venta)       AS ticket_promedio
FROM webinar.gold.ventas AS v
JOIN webinar.gold.tiendas AS t
  ON v.sk_tienda = t.sk_tienda
GROUP BY
  t.id_tienda,
  t.nombre,
  date_format(v.fecha_venta, 'yyyy-MM');

KPI: Clientes activos por mes

In [0]:
CREATE VIEW webinar.gold.clientes_activos
AS
SELECT
  date_format(v.fecha_venta, 'yyyy-MM') AS anio_mes,
  COUNT(DISTINCT v.sk_cliente)          AS kpi_clientes_activos
FROM webinar.gold.ventas v
WHERE date_format(v.fecha_venta, 'yyyy-MM') IS NOT NULL
GROUP BY
  date_format(v.fecha_venta, 'yyyy-MM');


Crecimiento mensual de ventas

In [0]:
CREATE VIEW webinar.gold.crecimiento_mensual
AS
WITH ventas_mensuales AS (
    SELECT
        date_format(fecha_venta, 'yyyy-MM') AS anio_mes,
        SUM(monto) AS monto_total
    FROM webinar.gold.ventas
    WHERE date_format(fecha_venta, 'yyyy-MM') IS NOT NULL
    GROUP BY date_format(fecha_venta, 'yyyy-MM')
)
SELECT
    anio_mes,
    monto_total,
    LAG(monto_total) OVER (ORDER BY anio_mes) AS monto_prev,
    (monto_total - LAG(monto_total) OVER (ORDER BY anio_mes))
        / LAG(monto_total) OVER (ORDER BY anio_mes) AS kpi_crecimiento_mom
FROM ventas_mensuales
ORDER BY anio_mes;
